# <center> Proposal for a Course Recommender System </center>


In [1]:
import csv
import copy
import pyodbc
import pickle
import implicit
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse as sparse

%matplotlib inline
plt.style.use('ggplot')


## 1. Load dataset as a Pandas dataframe

In [6]:
conn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=SRV25\MSSQL17_DEV;DATABASE=crme_test1;UID=sa_dev;PWD=rGiS$[8VmM')
query = '''
        select *
        from TrainingManagement.TrainingAssignedUsers
        '''

main_df = pd.read_sql(query, conn)
main_df.head(5)

,ID,PersonID,TrainingID,HotelRequired,HotelDesiredCheckIn,HotelDesiredCheckOut,HotelRequiredFromType,HotelMustHaveSmokingRoom,HotelMustHaveBreakfast,IsTravelRequired,...,VirtualRoomUserID,SubscriptionStartDate,SubscriptionEndDate,UpdateAt,UpdateBy,StatusTypeID,OrderID,OrderCancellationID,BookingID,CancellationID
0,1,3,312,True,NaT,NaT,1.0,True,False,True,...,NaN,NaT,NaT,NaT,NaN,0,NaN,NaN,161194,3.0
1,3,3,1925,False,NaT,NaT,1.0,False,False,True,...,NaN,NaT,NaT,NaT,NaN,0,NaN,NaN,177430,2.0
2,4,3,1631,False,NaT,NaT,1.0,False,False,True,...,NaN,NaT,NaT,NaT,NaN,0,NaN,NaN,169563,5.0
3,5,6,312,False,NaT,NaT,1.0,False,False,True,...,NaN,NaT,NaT,NaT,NaN,0,NaN,NaN,161195,4.0
4,7,30712,1388,False,NaT,NaT,1.0,False,False,False,...,NaN,NaT,NaT,NaT,NaN,0,NaN,NaN,178291,NaN


In [9]:
n_users = main_df['PersonID'].unique().shape[0]
n_courses = main_df['TrainingID'].unique().shape[0]

print(n_users, n_courses, main_df.shape)

26010 3497 (183619, 29)


In [10]:
def threshold_bookings(df, user_min, courses_min):
    n_users = df['PersonID'].unique().shape[0]
    n_courses = df['TrainingID'].unique().shape[0]
    
    done = False
    while not done:
        start_shape = df.shape[0]
        course_count = df.groupby('PersonID')['TrainingID'].count()
        df = df[~df['PersonID'].isin(course_count[course_count < courses_min].index.tolist())]
        user_count = df.groupby('TrainingID')['PersonID'].count()
        df = df[~df['TrainingID'].isin(user_count[user_count < user_min].index.tolist())]
        end_shape = df.shape[0]
        if start_shape == end_shape:
            done = True
    
    return df

main_df_lim = threshold_bookings(main_df, 5, 5)

In [18]:
course_count = main_df.groupby('PersonID')['TrainingID']
course_count.count().index

Int64Index([    1,     2,     3,     4,     5,     6,     7,    11,    14,
               16,
            ...
            50154, 50159, 50160, 50162, 50163, 50261, 50269, 50271, 50272,
            50310],
           dtype='int64', name='PersonID', length=26010)

In [11]:
main_df_lim.shape

(147207, 29)